In [1]:
%pip install duckdb pandas numpy pyspainmobility requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


<h1 align="center"><b>Building a 3-Tier Data Lakehouse for Mobility Analysis in Spain</b></h1>
<h3 align="center"><b style="color:gray">Bronze Layer</b></h3>
<h4 align="right">Joan Fernández Navarro & Borja Albert Gramaje</h4>
<h3><b>Table of Contents</b></h3>
<ul style = "list-style-type: none; line-height: 0.5em;">
    <li><a href="#mitma"><h5>1. Spanish Ministry of Transport, Mobility and Urban Agenda
(MITMA) Open Data</h5></a></li>
    <ul style = "list-style-type: none; line-height: 1em;">
        <li><a href="#od"><h5>1.1. Origin-destination (OD) trip matrices</h5></a></li>
        <li><a href="#people"><h5>1.2. People by day</h5></a></li>
        <li><a href="#overnight"><h5>1.3. Overnight stays</h5></a></li>
        <li><a href="#zones"><h5>1.4. Zones</h5></a></li>
    </ul>
    <li><a href="#ine"><h5>2. Spanish National Statistics Institute (INE)</h5></a></li>
    </ul>
</ul>

In [2]:
import os
import duckdb
import requests
import pandas as pd
from pyspainmobility import Mobility, Zones

BASE_PATH = f"{os.getcwd()}/../../raw"
LAKE_LAYER = "bronze"

con = duckdb.connect("./../../mobility.db")

def SQL(q):
    """Run SQL (printed for clarity) and return a DataFrame."""
    return con.execute(q).fetchdf()

print("DuckDB version:", con.sql("SELECT version();").fetchone()[0])

DuckDB version: v1.4.2


<h2 id="mitma"><b>1. Spanish Ministry of Transport, Mobility and Urban Agenda (MITMA) Open Data</b></h2>

<h2 id="od"><b>1.1. Origin-destination (OD) trip matrices</b></h2>

```sql
-- Trip Matrices - distristos
CREATE TABLE bronze_mitma_od_districts (
    fecha TEXT,
    periodo TEXT,
    origen TEXT,
    destino TEXT,
    distancia TEXT,
    actividad_origen TEXT,
    actividad_destino TEXT,
    residencia TEXT,
    renta TEXT,
    edad TEXT,
    sexo TEXT,
    viajes TEXT,
    viajes_km TEXT,
    -- Columnas extras añadidas para auditoria. 
    loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    source_file TEXT
);

-- Trip Matrices - municipalities
CREATE TABLE bronze_mitma_od_municipalities (
    fecha TEXT,
    periodo TEXT,
    origen TEXT,
    destino TEXT,
    distancia TEXT,
    actividad_origen TEXT,
    actividad_destino TEXT,
    residencia TEXT,
    renta TEXT,
    edad TEXT,
    sexo TEXT,
    viajes TEXT,
    viajes_km TEXT,
    -- Columnas extras añadidas para auditoria. 
    loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    source_file TEXT
);

-- Trip Matrices - GAU
CREATE TABLE bronze_mitma_od_gau (
    fecha TEXT,
    periodo TEXT,
    origen TEXT,
    destino TEXT,
    distancia TEXT,
    actividad_origen TEXT,
    actividad_destino TEXT,
    residencia TEXT,
    renta TEXT,
    edad TEXT,
    sexo TEXT,
    viajes TEXT,
    viajes_km TEXT,
    -- Columnas extras añadidas para auditoria. 
    loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    source_file TEXT
);
```

![Descripción de la imagen](./schemas/bronze_od.png)

In [3]:
def load_od_matrices(zone_type="districts", start_date="2022-03-01", end_date="2022-03-03"):
    """
    Downloads MITMA OD matrices (only if not already present) and loads them into DuckDB.

    Parameters
    ----------
    zone_type : str
        Zone level ("districts", "municipalities", etc.).
    start_date : str
        Start date (YYYY-MM-DD).
    end_date : str
        End date (YYYY-MM-DD).
    """

    dataset = "od"
    dataset_path = f"{BASE_PATH}/MITMA/{dataset}_{zone_type}"
    table_name = f"{LAKE_LAYER}_mitma_{dataset}_{zone_type}"

    # -------------------------------------------------------
    # 1. Ensure directory exists
    # -------------------------------------------------------
    os.makedirs(dataset_path, exist_ok=True)

    # -------------------------------------------------------
    # 2. Download data only if the directory is empty
    # -------------------------------------------------------
    if len(os.listdir(dataset_path)) == 0:
        print(f"Downloading MITMA OD matrices for zone type: {zone_type}...")
        mobility = Mobility(
            version=2,
            zones=zone_type,
            start_date=start_date,
            end_date=end_date,
            output_directory=str(dataset_path),
        )
        mobility.get_od_data(keep_activity=True)
    else:
        print("Files already exist. Skipping download.")

    # -------------------------------------------------------
    # 3. Create the target DuckDB table
    # -------------------------------------------------------
    SQL(f"""
        CREATE TABLE IF NOT EXISTS {table_name}(
            fecha TEXT,
            periodo TEXT,
            origen TEXT,
            destino TEXT,
            distancia TEXT,
            actividad_origen TEXT,
            actividad_destino TEXT,
            residencia TEXT,
            renta TEXT,
            edad TEXT,
            sexo TEXT,
            viajes TEXT,
            viajes_km TEXT,
            estudio_destino_posible TEXT,
            estudio_origen_posible TEXT,
            loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            source_file TEXT
        );
    """)

    # -------------------------------------------------------
    # 4. Load the CSV files into the table
    # -------------------------------------------------------
    SQL(f"""
        INSERT INTO {table_name}
        SELECT
            fecha,
            periodo,
            origen,
            destino,
            distancia,
            actividad_origen,
            actividad_destino,
            residencia,
            renta,
            edad,
            sexo,
            viajes,
            viajes_km,
            estudio_destino_posible,
            estudio_origen_posible,
            CURRENT_TIMESTAMP AS loaded_at,
            filename AS source_file
        FROM read_csv(
            '{dataset_path}/*.csv.gz',
            filename = true,
            all_varchar = true
        );
    """)

    print(f"Data successfully loaded into table: {table_name}")

load_od_matrices(zone_type="distritos")
load_od_matrices(zone_type="municipios")
load_od_matrices(zone_type="gau")

Downloading: https://movilidad-opendata.mitma.es/estudios_basicos/por-distritos/viajes/ficheros-diarios/2022-03/20220301_Viajes_distritos.csv.gz
Saved 178582407 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/od_distritos\20220301_Viajes_distritos_v2.csv.gz
Downloading: https://movilidad-opendata.mitma.es/estudios_basicos/por-distritos/viajes/ficheros-diarios/2022-03/20220302_Viajes_distritos.csv.gz
Saved 181734955 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/od_distritos\20220302_Viajes_distritos_v2.csv.gz
Downloading: https://movilidad-opendata.mitma.es/estudios_basicos/por-distritos/viajes/ficheros-diarios/2022-03/20220303_Viajes_distritos.csv.gz
Saved 182421127 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-p

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Processing file: C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/od_distritos\20220301_Viajes_distritos_v2.csv.gz
Reading gzipped file...


 33%|████████████████████████████                                                        | 1/3 [01:22<02:45, 82.92s/it]

Processing file: C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/od_distritos\20220302_Viajes_distritos_v2.csv.gz
Reading gzipped file...


...

Processing file: C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/od_gau\20220302_Viajes_GAU_v2.csv.gz
Reading gzipped file...


 67%|████████████████████████████████████████████████████████                            | 2/3 [01:20<00:39, 39.83s/it]

Processing file: C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/od_gau\20220303_Viajes_GAU_v2.csv.gz
Reading gzipped file...


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:53<00:00, 37.78s/it]


Concatenating all the dataframes....
Writing the parquet file....
Parquet file generated successfully at  C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/od_gau\Viajes_GAU_2022-03-01_2022-03-03_v2.parquet
Data successfully loaded into table: bronze_mitma_od_gau


In [4]:
SQL(f"""
    (SELECT '{LAKE_LAYER}_mitma_od_distritos' as name, count(*) 
    FROM {LAKE_LAYER}_mitma_od_distritos)
        UNION
    (SELECT '{LAKE_LAYER}_mitma_od_municipios' as name, count(*) 
    FROM {LAKE_LAYER}_mitma_od_municipios)
        UNION
    (SELECT '{LAKE_LAYER}_mitma_od_gau' as name, count(*) 
    FROM {LAKE_LAYER}_mitma_od_gau);
""")


,name,count_star()
0,bronze_mitma_od_distritos,55862966
1,bronze_mitma_od_gau,20827473
2,bronze_mitma_od_municipios,34684127


In [5]:
SQL(f"""
    SELECT * 
    FROM {LAKE_LAYER}_mitma_od_distritos 
    LIMIT 5;
""")

,fecha,periodo,origen,destino,distancia,actividad_origen,actividad_destino,residencia,renta,edad,sexo,viajes,viajes_km,estudio_destino_posible,estudio_origen_posible,loaded_at,source_file
0,20220301,08,01001,01002,10-50,casa,frecuente,01,10-15,NA,NA,2.764,125.486,no,no,2025-11-17 22:56:41.124366,C:\Users\joanf\OneDrive\Documentos\Estudios\MU...
1,20220301,17,01001,01002,10-50,no_frecuente,casa,01,10-15,NA,NA,6.526,303.751,no,no,2025-11-17 22:56:41.124366,C:\Users\joanf\OneDrive\Documentos\Estudios\MU...
2,20220301,00,01001,01009_AM,2-10,casa,frecuente,01,10-15,NA,NA,10.279,67.832,no,no,2025-11-17 22:56:41.124366,C:\Users\joanf\OneDrive\Documentos\Estudios\MU...
3,20220301,00,01001,01009_AM,2-10,frecuente,casa,01,>15,NA,NA,4.591,42.419,no,no,2025-11-17 22:56:41.124366,C:\Users\joanf\OneDrive\Documentos\Estudios\MU...
4,20220301,02,01001,01009_AM,2-10,casa,frecuente,01,10-15,NA,NA,2.539,13.819,no,no,2025-11-17 22:56:41.124366,C:\Users\joanf\OneDrive\Documentos\Estudios\MU...


<h2 id="people"><b>1.2. People by day</b></h2>

```sql
-- Distritos
CREATE TABLE IF NOT EXISTS bronze_mitma_people_day_districts (
  fecha TEXT,
  zona_pernoctacion TEXT,
  edad TEXT,
  sexo TEXT,
  numero_viajes TEXT,   -- 0,1,2,2+ (mantener TEXT)
  personas TEXT,
  -- Columnas extras añadidas para auditoria. 
  loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  source_file TEXT
);

-- Municipios
CREATE TABLE IF NOT EXISTS bronze_mitma_people_day_municipalities (
  fecha TEXT,
  zona_pernoctacion TEXT,
  edad TEXT,
  sexo TEXT,
  numero_viajes TEXT,
  personas TEXT,
  -- Columnas extras añadidas para auditoria. 
  loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  source_file TEXT
);

-- GAU
CREATE TABLE IF NOT EXISTS bronze_mitma_people_day_gau (
  fecha TEXT,
  zona_pernoctacion TEXT,
  edad TEXT,
  sexo TEXT,
  numero_viajes TEXT,
  personas TEXT,
  -- Columnas extras añadidas para auditoria. 
  loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  source_file TEXT
);
```

![Descripción de la imagen](./schemas/bronze_people_day.png)

In [6]:
def load_people_day(zone_type="districts", start_date="2022-03-01", end_date="2022-03-03"):
    """
    Downloads MITMA 'people_day' data (only if not already present)
    and loads it into DuckDB.

    Parameters
    ----------
    zone_type : str
        Zone level (“districts”, “municipalities”, etc.).
    start_date : str
        Start date (YYYY-MM-DD).
    end_date : str
        End date (YYYY-MM-DD).
    """

    dataset = "people_day"
    dataset_path = f"{BASE_PATH}/MITMA/{dataset}_{zone_type}"
    table_name = f"{LAKE_LAYER}_mitma_{dataset}_{zone_type}"

    # -------------------------------------------------------
    # 1. Ensure directory exists
    # -------------------------------------------------------
    os.makedirs(dataset_path, exist_ok=True)

    # -------------------------------------------------------
    # 2. Download data only if directory is empty
    # -------------------------------------------------------
    if len(os.listdir(dataset_path)) == 0:
        print(f"Downloading MITMA 'people_day' dataset for: {zone_type}...")
        mobility = Mobility(
            version=2,
            zones=zone_type,
            start_date=start_date,
            end_date=end_date,
            output_directory=str(dataset_path),
        )
        mobility.get_number_of_trips_data()
    else:
        print("Files already exist. Skipping download.")

    # -------------------------------------------------------
    # 3. Create DuckDB table
    # -------------------------------------------------------
    SQL(f"DROP TABLE IF EXISTS {table_name};")

    SQL(f"""
        CREATE TABLE {table_name}(
            fecha TEXT,
            zona_pernoctacion TEXT,
            edad TEXT,
            sexo TEXT,
            numero_viajes TEXT,
            personas TEXT,
            loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            source_file TEXT
        );
    """)

    # -------------------------------------------------------
    # 4. Load data from CSV into DuckDB
    # -------------------------------------------------------
    SQL(f"""
        INSERT INTO {table_name}
        SELECT
            fecha,
            zona_pernoctacion,
            edad,
            sexo,
            numero_viajes,
            personas,
            CURRENT_TIMESTAMP AS loaded_at,
            filename AS source_file
        FROM read_csv(
            '{dataset_path}/*.csv.gz',
            filename = true,
            all_varchar = true
        );
    """)

    print(f"Data successfully loaded into table: {table_name}")


load_people_day(zone_type="distritos")
load_people_day(zone_type="municipios")
load_people_day(zone_type="gau")

Downloading: https://movilidad-opendata.mitma.es/estudios_basicos/por-distritos/personas/ficheros-diarios/2022-03/20220301_Personas_dia_distritos.csv.gz
Saved 784312 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/people_day_distritos\20220301_Personas_distritos_v2.csv.gz
Downloading: https://movilidad-opendata.mitma.es/estudios_basicos/por-distritos/personas/ficheros-diarios/2022-03/20220302_Personas_dia_distritos.csv.gz
Saved 774109 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/people_day_distritos\20220302_Personas_distritos_v2.csv.gz
Downloading: https://movilidad-opendata.mitma.es/estudios_basicos/por-distritos/personas/ficheros-diarios/2022-03/20220303_Personas_dia_distritos.csv.gz
Saved 773512 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineeri

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  4.29it/s]


Concatenating all the dataframes....
Writing the parquet file....
Parquet file generated successfully at  C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/people_day_distritos\Personas_distritos_2022-03-01_2022-03-03_v2.parquet
Data successfully loaded into table: bronze_mitma_people_day_distritos
Downloading: https://movilidad-opendata.mitma.es/estudios_basicos/por-municipios/personas/ficheros-diarios/2022-03/20220301_Personas_dia_municipios.csv.gz
Saved 530935 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/people_day_municipios\20220301_Personas_municipios_v2.csv.gz
Downloading: https://movilidad-opendata.mitma.es/estudios_basicos/por-municipios/personas/ficheros-diarios/2022-03/20220302_Personas_dia_municipios.csv.gz
Saved 522530 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUC

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  5.99it/s]


Concatenating all the dataframes....
Writing the parquet file....
Parquet file generated successfully at  C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/people_day_municipios\Personas_municipios_2022-03-01_2022-03-03_v2.parquet
Data successfully loaded into table: bronze_mitma_people_day_municipios
Downloading: https://movilidad-opendata.mitma.es/estudios_basicos/por-GAU/personas/ficheros-diarios/2022-03/20220301_Personas_dia_GAU.csv.gz
Saved 418238 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/people_day_gau\20220301_Personas_GAU_v2.csv.gz
Downloading: https://movilidad-opendata.mitma.es/estudios_basicos/por-GAU/personas/ficheros-diarios/2022-03/20220302_Personas_dia_GAU.csv.gz
Saved 410613 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologi

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  7.03it/s]


Concatenating all the dataframes....
Writing the parquet file....
Parquet file generated successfully at  C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/people_day_gau\Personas_GAU_2022-03-01_2022-03-03_v2.parquet
Data successfully loaded into table: bronze_mitma_people_day_gau


In [7]:
SQL(f"""
    (SELECT '{LAKE_LAYER}_mitma_people_day_distritos' as name, count(*) 
    FROM {LAKE_LAYER}_mitma_people_day_distritos)
        UNION
    (SELECT '{LAKE_LAYER}_mitma_people_day_municipios' as name, count(*) 
    FROM {LAKE_LAYER}_mitma_people_day_municipios)
        UNION
    (SELECT '{LAKE_LAYER}_mitma_people_day_gau' as name, count(*) 
    FROM {LAKE_LAYER}_mitma_people_day_gau);
""")

,name,count_star()
0,bronze_mitma_people_day_distritos,340448
1,bronze_mitma_people_day_gau,182214
2,bronze_mitma_people_day_municipios,231688


In [8]:
SQL(f"""
    SELECT * 
    FROM {LAKE_LAYER}_mitma_people_day_distritos 
    LIMIT 5;
""")

,fecha,zona_pernoctacion,edad,sexo,numero_viajes,personas,loaded_at,source_file
0,20220301,01001,0-25,hombre,0,125.296,2025-11-17 23:09:35.219077,C:\Users\joanf\OneDrive\Documentos\Estudios\MU...
1,20220301,01001,0-25,hombre,2,115.378,2025-11-17 23:09:35.219077,C:\Users\joanf\OneDrive\Documentos\Estudios\MU...
2,20220301,01001,0-25,hombre,2+,176.630,2025-11-17 23:09:35.219077,C:\Users\joanf\OneDrive\Documentos\Estudios\MU...
3,20220301,01001,0-25,mujer,0,125.069,2025-11-17 23:09:35.219077,C:\Users\joanf\OneDrive\Documentos\Estudios\MU...
4,20220301,01001,0-25,mujer,2,117.712,2025-11-17 23:09:35.219077,C:\Users\joanf\OneDrive\Documentos\Estudios\MU...


<h2 id="overnight"><b>1.3. Overnight stays</b></h2>

```sql
-- Distritos
CREATE TABLE IF NOT EXISTS bronze_mitma_overnight_stay_districts (
  fecha TEXT,
  zona_residencia TEXT,
  zona_pernoctacion TEXT,
  personas TEXT,
  -- Columnas extras añadidas para auditoria. 
  loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  source_file TEXT
);

-- Municipios
CREATE TABLE IF NOT EXISTS bronze_mitma_overnight_stay_municipalities (
  fecha TEXT,
  zona_residencia TEXT,
  zona_pernoctacion TEXT,
  personas TEXT,
  -- Columnas extras añadidas para auditoria. 
  loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  source_file TEXT
);

-- GAU
CREATE TABLE IF NOT EXISTS bronze_mitma_overnight_stay_gau (
  fecha TEXT,
  zona_residencia TEXT,
  zona_pernoctacion TEXT,
  personas TEXT,
  -- Columnas extras añadidas para auditoria. 
  loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  source_file TEXT
);
```

![Descripción de la imagen](./schemas/bronze_overnight.png)

In [9]:
def load_overnight_stay(zone_type="districts", start_date="2022-03-01", end_date="2022-03-03"):
    """
    Downloads MITMA overnight-stay data (only if missing) and loads it into DuckDB.

    Parameters
    ----------
    zone_type : str
        Zone level (“districts”, “municipalities”, etc.).
    start_date : str
        Start date (YYYY-MM-DD).
    end_date : str
        End date (YYYY-MM-DD).
    """

    dataset = "overnight_stay"
    dataset_path = f"{BASE_PATH}/MITMA/{dataset}_{zone_type}"
    table_name = f"{LAKE_LAYER}_mitma_{dataset}_{zone_type}"

    # -------------------------------------------------------
    # 1. Ensure directory exists
    # -------------------------------------------------------
    os.makedirs(dataset_path, exist_ok=True)

    # -------------------------------------------------------
    # 2. Download data only if directory is empty
    # -------------------------------------------------------
    if len(os.listdir(str(dataset_path))) == 0:
        print(f"Downloading MITMA overnight-stay data for zone type: {zone_type}...")
        mobility = Mobility(
            version=2,
            zones=zone_type,
            start_date=start_date,
            end_date=end_date,
            output_directory=str(dataset_path),
        )
        mobility.get_overnight_stays_data()
    else:
        print("Files already exist. Skipping download.")

    # -------------------------------------------------------
    # 3. Create table in DuckDB
    # -------------------------------------------------------
    SQL(f"DROP TABLE IF EXISTS {table_name};")

    SQL(f"""
        CREATE TABLE {table_name}(
            fecha TEXT,
            zona_residencia TEXT,
            zona_pernoctacion TEXT,
            personas TEXT,
            loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            source_file TEXT
        );
    """)

    # -------------------------------------------------------
    # 4. Insert CSV data
    # -------------------------------------------------------
    SQL(f"""
        INSERT INTO {table_name}
        SELECT
            fecha,
            zona_residencia,
            zona_pernoctacion,
            personas,
            CURRENT_TIMESTAMP AS loaded_at,
            filename AS source_file
        FROM read_csv(
            '{dataset_path}/*.csv.gz',
            filename = true,
            all_varchar = true
        );
    """)

    print(f"Overnight-stay data successfully loaded into table: {table_name}")


load_overnight_stay(zone_type="distritos")
load_overnight_stay(zone_type="municipios")
load_overnight_stay(zone_type="gau")

Downloading: https://movilidad-opendata.mitma.es/estudios_basicos/por-distritos/pernoctaciones/ficheros-diarios/2022-03/20220301_Pernoctaciones_distritos.csv.gz
Saved 1853342 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/overnight_stay_distritos\20220301_Pernoctaciones_distritos_v2.csv.gz
Downloading: https://movilidad-opendata.mitma.es/estudios_basicos/por-distritos/pernoctaciones/ficheros-diarios/2022-03/20220302_Pernoctaciones_distritos.csv.gz
Saved 1643373 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/overnight_stay_distritos\20220302_Pernoctaciones_distritos_v2.csv.gz
Downloading: https://movilidad-opendata.mitma.es/estudios_basicos/por-distritos/pernoctaciones/ficheros-diarios/2022-03/20220303_Pernoctaciones_distritos.csv.gz
Saved 1623621 bytes to C:\Users\joanf\OneDriv

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.61it/s]


Concatenating all the dataframes....
Writing the parquet file....
Parquet file generated successfully at  C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/overnight_stay_distritos\Pernoctaciones_distritos_2022-03-01_2022-03-03_v2.parquet
Overnight-stay data successfully loaded into table: bronze_mitma_overnight_stay_distritos
Downloading: https://movilidad-opendata.mitma.es/estudios_basicos/por-municipios/pernoctaciones/ficheros-diarios/2022-03/20220301_Pernoctaciones_municipios.csv.gz
Saved 1468082 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/overnight_stay_municipios\20220301_Pernoctaciones_municipios_v2.csv.gz
Downloading: https://movilidad-opendata.mitma.es/estudios_basicos/por-municipios/pernoctaciones/ficheros-diarios/2022-03/20220302_Pernoctaciones_municipios.csv.gz
Saved 1297818

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  4.83it/s]


Concatenating all the dataframes....
Writing the parquet file....
Parquet file generated successfully at  C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/overnight_stay_municipios\Pernoctaciones_municipios_2022-03-01_2022-03-03_v2.parquet
Overnight-stay data successfully loaded into table: bronze_mitma_overnight_stay_municipios
Downloading: https://movilidad-opendata.mitma.es/estudios_basicos/por-GAU/pernoctaciones/ficheros-diarios/2022-03/20220301_Pernoctaciones_GAU.csv.gz
Saved 1255035 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/overnight_stay_gau\20220301_Pernoctaciones_GAU_v2.csv.gz
Downloading: https://movilidad-opendata.mitma.es/estudios_basicos/por-GAU/pernoctaciones/ficheros-diarios/2022-03/20220302_Pernoctaciones_GAU.csv.gz
Saved 1099165 bytes to C:\Users\joanf\OneDrive\Docum

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  4.82it/s]


Concatenating all the dataframes....
Writing the parquet file....
Parquet file generated successfully at  C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/overnight_stay_gau\Pernoctaciones_GAU_2022-03-01_2022-03-03_v2.parquet
Overnight-stay data successfully loaded into table: bronze_mitma_overnight_stay_gau


In [10]:
SQL(f"""
    (SELECT '{LAKE_LAYER}_mitma_overnight_stay_distritos' as name, count(*) 
    FROM {LAKE_LAYER}_mitma_overnight_stay_distritos)
        UNION
    (SELECT '{LAKE_LAYER}_mitma_overnight_stay_municipios' as name, count(*) 
    FROM {LAKE_LAYER}_mitma_overnight_stay_municipios)
        UNION
    (SELECT '{LAKE_LAYER}_mitma_overnight_stay_gau' as name, count(*) 
    FROM {LAKE_LAYER}_mitma_overnight_stay_gau);
""")

,name,count_star()
0,bronze_mitma_overnight_stay_municipios,741266
1,bronze_mitma_overnight_stay_distritos,947839
2,bronze_mitma_overnight_stay_gau,574703


In [11]:
SQL(f"""
    SELECT * 
    FROM {LAKE_LAYER}_mitma_overnight_stay_distritos 
    LIMIT 5;
""")

,fecha,zona_residencia,zona_pernoctacion,personas,loaded_at,source_file
0,20220301,01001,01001,2733.784,2025-11-17 23:09:51.583001,C:\Users\joanf\OneDrive\Documentos\Estudios\MU...
1,20220301,01001,01004_AM,2.514,2025-11-17 23:09:51.583001,C:\Users\joanf\OneDrive\Documentos\Estudios\MU...
2,20220301,01001,01009_AM,18.431,2025-11-17 23:09:51.583001,C:\Users\joanf\OneDrive\Documentos\Estudios\MU...
3,20220301,01001,01017_AM,2.922,2025-11-17 23:09:51.583001,C:\Users\joanf\OneDrive\Documentos\Estudios\MU...
4,20220301,01001,01051,7.831,2025-11-17 23:09:51.583001,C:\Users\joanf\OneDrive\Documentos\Estudios\MU...


<h2 id="zones"><b>1.4. Zones</b></h2>

```sql
-- Distritos
CREATE TABLE IF NOT EXISTS bronze_mitma_districts (
  id TEXT,
  name TEXT,
  population TEXT,
  geometry TEXT,
  -- Columnas extras añadidas para auditoria. 
  loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  source_file TEXT
);

-- Municipios
CREATE TABLE IF NOT EXISTS bronze_mitma_municipalities (
  id TEXT,
  name TEXT,
  population TEXT,
  geometry TEXT,
  -- Columnas extras añadidas para auditoria. 
  loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  source_file TEXT
);

-- GAU
CREATE TABLE IF NOT EXISTS bronze_mitma_gau (
  id TEXT,
  name TEXT,
  population TEXT,
  geometry TEXT,
  -- Columnas extras añadidas para auditoria. 
  loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  source_file TEXT
);

```

![Descripción de la imagen](./schemas/bronze_zones.png)

In [12]:
def load_zones(zone_type="districts"):
    """
    Downloads MITMA zone definitions (only if missing), stores them as compressed CSV,
    and loads them into DuckDB.

    Parameters
    ----------
    zone_type : str
        Zone level (“districts”, “municipalities”, “gau”, etc.).
    """

    dataset = "zones"
    dataset_path = f"{BASE_PATH}/MITMA/{zone_type}"
    table_name = f"{LAKE_LAYER}_mitma_{zone_type}"

    # -------------------------------------------------------
    # 1. Ensure directory exists
    # -------------------------------------------------------
    os.makedirs(dataset_path, exist_ok=True)

    csv_path = dataset_path + "/zones.csv.gz"

    # -------------------------------------------------------
    # 2. Download and save zones CSV if not present
    # -------------------------------------------------------
    if not os.path.isfile(dataset_path):
        print(f"Downloading MITMA zone definitions for zone type: {zone_type}...")

        zones = Zones(
            version=2,
            zones=zone_type,
            output_directory=str(dataset_path),
        )

        df = zones.get_zone_geodataframe()

        if df is None:
            raise ValueError("Zones.get_zone_geodataframe() returned None")

        # Save geodataframe as compressed CSV
        df.to_csv(csv_path, index=True, compression="gzip")
        print("Zones saved:", csv_path)
    
    else:
        print("Zone definition file already exists. Skipping download.")

    # -------------------------------------------------------
    # 3. Create DuckDB table
    # -------------------------------------------------------
    SQL(f"DROP TABLE IF EXISTS {table_name};")

    SQL(f"""
        CREATE TABLE {table_name}(
            id TEXT,
            name TEXT,
            population TEXT,
            geometry TEXT,   -- stored as plain text in BRONZE layer
            loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            source_file TEXT
        );
    """)

    # -------------------------------------------------------
    # 4. Load CSV into DuckDB
    # -------------------------------------------------------
    SQL(f"""
        INSERT INTO {table_name}
        SELECT
            id,
            name,
            population,
            geometry,
            CURRENT_TIMESTAMP AS loaded_at,
            filename AS source_file
        FROM read_csv(
            '{dataset_path}/*.csv.gz',
            filename = true,
            all_varchar = true
        );
    """)

    print(f"Zone data successfully loaded into table: {table_name}")



load_zones(zone_type="distritos")
load_zones(zone_type="municipios")
load_zones(zone_type="gau")

Downloading: https://movilidad-opendata.mitma.es/zonificacion/poblacion.csv
Saved 97707 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/distritos\poblacion.csv
Downloading: https://movilidad-opendata.mitma.es/zonificacion/relacion_ine_zonificacionMitma.csv
Saved 1919214 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/distritos\relacion_ine_zonificacionMitma.csv
Downloading: https://movilidad-opendata.mitma.es/zonificacion/zonificacion_distritos/zonificacion_distritos_centroides.qpj
Saved 618 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/distritos\zonificacion_distritos_centroides.qpj
Downloading: https://movilidad-opendata.mitma.es/zonificacion/zonificacion_distritos/zonific

C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\.venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/distritos\zonificacion_distritos.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


Zones saved: C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/distritos/zones.csv.gz
Zone data successfully loaded into table: bronze_mitma_distritos
Downloading: https://movilidad-opendata.mitma.es/zonificacion/poblacion.csv
Saved 97707 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/municipios\poblacion.csv
Downloading: https://movilidad-opendata.mitma.es/zonificacion/relacion_ine_zonificacionMitma.csv
Saved 1919214 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/municipios\relacion_ine_zonificacionMitma.csv
Downloading: https://movilidad-opendata.mitma.es/zonificacion/zonificacion_municipios/zonificacion_municipios_centroides.prj
Saved 393 bytes to C:\Users\joanf\OneDrive\Documentos\

C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\.venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/municipios\zonificacion_municipios.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


Zones saved: C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/municipios/zones.csv.gz
Zone data successfully loaded into table: bronze_mitma_municipios
Downloading: https://movilidad-opendata.mitma.es/zonificacion/poblacion.csv
Saved 97707 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/gau\poblacion.csv
Downloading: https://movilidad-opendata.mitma.es/zonificacion/relacion_ine_zonificacionMitma.csv
Saved 1919214 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/gau\relacion_ine_zonificacionMitma.csv
Downloading: https://movilidad-opendata.mitma.es/zonificacion/zonificacion_GAU/zonificacion_gaus_centroides.cpg
Saved 5 bytes to C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data En

C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\.venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/gau\zonificacion_gaus.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


Zones saved: C:\Users\joanf\OneDrive\Documentos\Estudios\MUCEIM\Big Data Engineering and Technologies\Project\borja-project\notebooks\layers/../../raw/MITMA/gau/zones.csv.gz
Zone data successfully loaded into table: bronze_mitma_gau


In [13]:
SQL(f"""
    (SELECT '{LAKE_LAYER}_mitma_distritos' as name, count(*) 
    FROM {LAKE_LAYER}_mitma_distritos)
        UNION
    (SELECT '{LAKE_LAYER}_mitma_municipios' as name, count(*) 
    FROM {LAKE_LAYER}_mitma_municipios)
        UNION
    (SELECT '{LAKE_LAYER}_mitma_gau' as name, count(*)
    FROM {LAKE_LAYER}_mitma_gau);
""")

,name,count_star()
0,bronze_mitma_municipios,2618
1,bronze_mitma_gau,2086
2,bronze_mitma_distritos,3792


In [14]:
SQL(f"""
    SELECT * 
    FROM {LAKE_LAYER}_mitma_od_distritos 
    LIMIT 5;
""")

,fecha,periodo,origen,destino,distancia,actividad_origen,actividad_destino,residencia,renta,edad,sexo,viajes,viajes_km,estudio_destino_posible,estudio_origen_posible,loaded_at,source_file
0,20220301,08,01001,01002,10-50,casa,frecuente,01,10-15,NA,NA,2.764,125.486,no,no,2025-11-17 22:56:41.124366,C:\Users\joanf\OneDrive\Documentos\Estudios\MU...
1,20220301,17,01001,01002,10-50,no_frecuente,casa,01,10-15,NA,NA,6.526,303.751,no,no,2025-11-17 22:56:41.124366,C:\Users\joanf\OneDrive\Documentos\Estudios\MU...
2,20220301,00,01001,01009_AM,2-10,casa,frecuente,01,10-15,NA,NA,10.279,67.832,no,no,2025-11-17 22:56:41.124366,C:\Users\joanf\OneDrive\Documentos\Estudios\MU...
3,20220301,00,01001,01009_AM,2-10,frecuente,casa,01,>15,NA,NA,4.591,42.419,no,no,2025-11-17 22:56:41.124366,C:\Users\joanf\OneDrive\Documentos\Estudios\MU...
4,20220301,02,01001,01009_AM,2-10,casa,frecuente,01,10-15,NA,NA,2.539,13.819,no,no,2025-11-17 22:56:41.124366,C:\Users\joanf\OneDrive\Documentos\Estudios\MU...


<h2 id="ine"><b>2. Spanish National Statistics Institute (INE)</b></h2>

```sql
-- Distritos
CREATE TABLE IF NOT EXISTS bronze_ine_padron_municipios (
  cod        VARCHAR,
  nombre     VARCHAR,
  fk_unidad  INTEGER,
  fk_escala  INTEGER,
  data_txt   TEXT,
  data       JSON,
  -- Columnas extras añadidas para auditoria. 
  loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  -- loaded_by TEXT DEFAULT CURRENT_USER,
  source_file TEXT
);
```

![Descripción de la imagen](./schemas/bronze_ine.png)

In [15]:
def get_padron_by_municipio(year: int):
    """
    Fetches Spanish municipal population (Padrón) data from INE's WS Tempus API.

    Parameters
    ----------
    year : int
        Year for which the Padrón data is requested.

    Returns
    -------
    pandas.DataFrame
        Normalized JSON response as DataFrame.
    """

    url = f"https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/29005?date={year}0101:{year}1231"

    response = requests.get(url, timeout=120)
    response.raise_for_status()

    data = response.json()
    df = pd.json_normalize(data)
    return df


def load_padron(year=2023):
    """
    Downloads INE municipal Padrón data (only if missing) and loads it into DuckDB.

    Parameters
    ----------
    year : int
        Year of the Padrón dataset to retrieve.
    """

    dataset = "padron_municipios"
    dataset_path = f"{BASE_PATH}/INE/{dataset}"
    table_name = f"{LAKE_LAYER}_ine_{dataset}"

    filename = f"padron_municipios_{year}.csv.gz"
    file_path = dataset_path / filename

    # -------------------------------------------------------
    # 1. Ensure directory exists
    # -------------------------------------------------------
    dataset_path.mkdir(parents=True, exist_ok=True)

    # -------------------------------------------------------
    # 2. Download and save CSV if not present
    # -------------------------------------------------------
    if not file_path.exists():
        print(f"Downloading INE Padrón municipal data for year {year}...")

        df = get_padron_by_municipio(year)

        if df is not None and not df.empty:
            df.to_csv(file_path, index=False, compression="gzip")
        else:
            raise ValueError("get_padron_by_municipio() returned an empty or null dataframe")
    else:
        print("File already exists. Skipping download.")

    # -------------------------------------------------------
    # 3. Create DuckDB table
    # -------------------------------------------------------
    SQL(f"DROP TABLE IF EXISTS {table_name};")

    SQL(f"""
        CREATE TABLE {table_name}(
            cod        VARCHAR,
            nombre     VARCHAR,
            fk_unidad  INTEGER,
            fk_escala  INTEGER,
            data_txt   TEXT,
            data       JSON,
            loaded_at  TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            source_file TEXT
        );
    """)

    # -------------------------------------------------------
    # 4. Insert CSV data into DuckDB
    # -------------------------------------------------------
    SQL(f"""
        INSERT INTO {table_name}
        SELECT
            COD        AS cod,
            Nombre     AS nombre,
            FK_Unidad::INTEGER AS fk_unidad,
            FK_Escala::INTEGER AS fk_escala,
            REGEXP_REPLACE(
                REGEXP_REPLACE(
                    REGEXP_REPLACE(
                        REPLACE(data, '''', '"'),
                        '\\bTrue\\b', 'true'
                    ),
                    '\\bFalse\\b', 'false'
                ),
                '\\bNone\\b', 'null'
            ) AS data_txt,
            CAST(data_txt AS JSON) AS data,
            CURRENT_TIMESTAMP AS loaded_at,
            filename AS source_file
        FROM read_csv(
            '{file_path}',
            filename = true,
            all_varchar = true
        );
    """)

    print(f"INE Padrón data for year {year} successfully loaded into table: {table_name}")


load_padron(2023)

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [ ]:
SQL("""
    SELECT * 
    FROM bronze_ine_padron_municipios 
    LIMIT 5;
""")